In [1]:
import numpy as np
weights = np.load("weights.npy", allow_pickle=True).item()

In [2]:
from gguf import GGUFWriter

architecture = "purpledetector"
gguf = GGUFWriter("weights.gguf", architecture)

gguf.add_name("weights")
gguf.add_description("description here")

gguf.add_uint32("input_width", 64)
gguf.add_uint32("layer_count", 2)

for k, v in weights.items():
    gguf.add_tensor(k, v)

gguf.write_header_to_file()
gguf.write_kv_data_to_file()
gguf.write_tensors_to_file()
gguf.close()

In [3]:
# To avoid having to load png and resize in C we'll just export to float binaries
import glob
from PIL import Image

for f in glob.glob("*/*/*.png"):
    image = Image.open(f).convert('RGB')
    image = image.resize((64, 128))
    image = np.array(image) / 255.0
    image = image.reshape(128, -1)
    image = image.astype(np.float32)
    
    with open(f.replace(".png", ".bin"), "wb") as out_f:
        image.tofile(out_f)